In [ ]:
!pip install pyspark


In [9]:
import os

from pyspark.sql import SparkSession,Row
import sqlite3

In [13]:


con = sqlite3.connect('example.db')
cur = con.cursor()
# Create table
cur.execute(
    '''CREATE TABLE IF NOT EXISTS stocks
       (date text, trans text, symbol text, qty real, price real)''')
# Insert a row of data
cur.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")
# Save (commit) the changes
con.commit()
con.close()



In [ ]:
spark = (SparkSession.builder
    .master("local")
    .appName("SQLite JDBC")
    .config(
        "spark.jars",
        "/Users/derib/data_architecture/sqlite-jdbc-3.34.0.jar".format(os.getcwd()))
    .config(
        "spark.driver.extraClassPath",
        "/Users/derib/data_architecture/sqlite-jdbc-3.34.0.jar".format(os.getcwd()))
    .getOrCreate())
spark

In [15]:
df = spark.read.format('jdbc') \
          .options(driver='org.sqlite.JDBC', dbtable='stocks',
                   url='jdbc:sqlite:/Users/derib/data_architecture/tp_spark/example.db').load()

In [16]:
df.show()

+----------+-----+------+-----+-----+
|      date|trans|symbol|  qty|price|
+----------+-----+------+-----+-----+
|2006-01-05|  BUY|  RHAT|100.0|35.14|
|2006-01-05|  BUY|  RHAT|100.0|35.14|
+----------+-----+------+-----+-----+



In [17]:
df_other = spark.read.jdbc(url='jdbc:sqlite:/Users/derib/data_architecture/tp_spark/example.db',table="stocks")

In [18]:


# Create dummy data
data = [
    Row(date="2023-01-01", trans="BUY", symbol="AAPL", qty=50, price=150.0),
    Row(date="2023-01-02", trans="SELL", symbol="GOOG", qty=30, price=2800.0),
    Row(date="2023-01-03", trans="BUY", symbol="MSFT", qty=20, price=300.0),
    Row(date="2023-01-04", trans="SELL", symbol="AMZN", qty=10, price=3500.0)
]

# Create Spark DataFrame
dummy_df = spark.createDataFrame(data)

# Show the DataFrame
dummy_df.show()

+----------+-----+------+---+------+
|      date|trans|symbol|qty| price|
+----------+-----+------+---+------+
|2023-01-01|  BUY|  AAPL| 50| 150.0|
|2023-01-02| SELL|  GOOG| 30|2800.0|
|2023-01-03|  BUY|  MSFT| 20| 300.0|
|2023-01-04| SELL|  AMZN| 10|3500.0|
+----------+-----+------+---+------+



In [21]:
# Write the DataFrame to SQLite
dummy_df.write \
    .format('jdbc') \
    .options(driver='org.sqlite.JDBC', dbtable='stocks',
             url='jdbc:sqlite:/Users/derib/data_architecture/tp_spark/example.db') \
    .mode('append') \
    .save()

25/04/07 15:39:52 WARN JdbcUtils: Requested isolation level 1 is not supported; falling back to default isolation level 8
